Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
batch_size = 400

hidden_nodes = 5000

graph = tf.Graph()
with graph.as_default():
    
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size \
                                        * image_size))
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_beta = tf.constant(0.001)
    
    #variables: input layer  
    input_layer = {'weights': \
    tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
    'biases': tf.Variable(tf.zeros([hidden_nodes]))}

    #hidden layer
    hidden_layer = {'weights':\
    tf.Variable(tf.truncated_normal([hidden_nodes,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
 
    #This is the network, given a dataset X it will return
    #the score after the hidden layer:
    # W1*X + b1 ---relu---> W2*Y + b2
    def network(data):
        inp_la = tf.matmul(data, input_layer['weights'])\
         + input_layer['biases'] 
        relu = tf.nn.relu(inp_la)
        logits = tf.matmul(relu, hidden_layer['weights'])\
         + hidden_layer['biases']
        return logits
   
        
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels)) \
        + tf_beta*tf.nn.l2_loss(input_layer['weights'])\
        + tf_beta*tf.nn.l2_loss(hidden_layer['weights'])
  
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
        
    
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 1.773189
Minibatch accuracy: 36.8%
Validation accuracy: 6.2%
Minibatch loss at step 500: 945.248596
Minibatch accuracy: 86.2%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 558.286438
Minibatch accuracy: 89.8%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 336.816101
Minibatch accuracy: 91.5%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 203.860321
Minibatch accuracy: 93.5%
Validation accuracy: 86.8%
Minibatch loss at step 2500: 123.327858
Minibatch accuracy: 92.8%
Validation accuracy: 87.5%
Minibatch loss at step 3000: 74.783775
Minibatch accuracy: 97.5%
Validation accuracy: 88.1%
Test accuracy: 94.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [20]:
batch_size = 12

hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size \
                                        * image_size))
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_beta = tf.constant(0.001)
    
    #variables: input layer  
    input_layer = {'weights': \
    tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
    'biases': tf.Variable(tf.zeros([hidden_nodes]))}

    #hidden layer
    hidden_layer = {'weights':\
    tf.Variable(tf.truncated_normal([hidden_nodes,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
 
    #This is the network, given a dataset X it will return
    #the score after the hidden layer:
    # W1*X + b1 ---relu---> W2*Y + b2
    def network(data):
        inp_la = tf.matmul(data, input_layer['weights'])\
         + input_layer['biases'] 
        relu = tf.nn.relu(inp_la)
        logits = tf.matmul(relu, hidden_layer['weights'])\
         + hidden_layer['biases']
        return logits
   
        
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels)) \
        + tf_beta*tf.nn.l2_loss(input_layer['weights'])\
        + tf_beta*tf.nn.l2_loss(hidden_layer['weights'])
  
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 549.094482
Minibatch accuracy: 0.0%
Validation accuracy: 21.7%
Minibatch loss at step 500: 1128.771606
Minibatch accuracy: 66.7%
Validation accuracy: 58.6%
Minibatch loss at step 1000: 575.890198
Minibatch accuracy: 66.7%
Validation accuracy: 56.8%
Minibatch loss at step 1500: 467.617310
Minibatch accuracy: 41.7%
Validation accuracy: 48.9%
Minibatch loss at step 2000: 197.744904
Minibatch accuracy: 41.7%
Validation accuracy: 50.3%
Minibatch loss at step 2500: 117.723328
Minibatch accuracy: 75.0%
Validation accuracy: 61.1%
Minibatch loss at step 3000: 71.191589
Minibatch accuracy: 66.7%
Validation accuracy: 65.4%
Test accuracy: 71.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [69]:
#USING ONLY THE DROPOUTFUNCTION

batch_size = 12

hidden_nodes = 1024


graph = tf.Graph()
with graph.as_default():
    
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size \
                                        * image_size),name='train')
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables: input layer  
    input_layer = {'weights': \
    tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
    'biases': tf.Variable(tf.zeros([hidden_nodes]))}

    #hidden layer
    hidden_layer = {'weights':\
    tf.Variable(tf.truncated_normal([hidden_nodes,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
 
    #This is the network, given a dataset X it will return
    #the score after the hidden layer:
    # W1*X + b1 ---relu---> W2*Y + b2
    def network(data):
        inp_la = tf.matmul(data, input_layer['weights'])\
         + input_layer['biases']
        if data.name=='train:0':
                probability = tf.constant(0.78, dtype='float32')
                relu = tf.nn.relu(tf.nn.dropout(inp_la,probability))
        else:
            relu = tf.nn.relu(inp_la) 
        logits = tf.matmul(relu, hidden_layer['weights'])\
         + hidden_layer['biases']
        return logits
   
        
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels))
  
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 425.165039
Minibatch accuracy: 0.0%
Validation accuracy: 21.4%
Minibatch loss at step 500: 182889296.000000
Minibatch accuracy: 66.7%
Validation accuracy: 45.8%
Minibatch loss at step 1000: 232162997567488.000000
Minibatch accuracy: 50.0%
Validation accuracy: 42.8%
Minibatch loss at step 1500: 45568916365548453888.000000
Minibatch accuracy: 25.0%
Validation accuracy: 49.7%
Minibatch loss at step 2000: 1862573946075540688470016.000000
Minibatch accuracy: 33.3%
Validation accuracy: 45.0%
Minibatch loss at step 2500: 197873256929734002035478495232.000000
Minibatch accuracy: 25.0%
Validation accuracy: 47.4%
Minibatch loss at step 3000: 1948215100238088187886543055093760.000000
Minibatch accuracy: 58.3%
Validation accuracy: 51.1%
Test accuracy: 54.2%


---
I did some tests with the dropout function:
---------

0) With probability 0.8916 we have a test accuracy of 88.8%

1) With probability 0.78 we have a test accuracy of 89.0%

2) With probability 0.6 we have a test accuracy of 87.2%

3) With probability 0.543 we have a test accuracy of 86.7%

4) With probability 0.25 we have a test accuracy of 76.2%

5) With probability 0.1 we have a test accuracy of 77.5%

6) With probability 0.05 we have a test accuracy of 10.0%


----remember the normal NN have a test accuracy of 88.5%



In [48]:
#DROPOUTFUNCTION WITH RANDOM PROBABILITIES

batch_size = 128

hidden_nodes = 1024


graph = tf.Graph()
with graph.as_default():
    
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size \
                                        * image_size),name='train')
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))
    
    #probability to be used in the dropout function 
    tf_probability = tf.placeholder(tf.float32, shape=())

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables: input layer  
    input_layer = {'weights': \
    tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
    'biases': tf.Variable(tf.zeros([hidden_nodes]))}

    #hidden layer
    hidden_layer = {'weights':\
    tf.Variable(tf.truncated_normal([hidden_nodes,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
 
    #This is the network, given a dataset X it will return
    #the score after the hidden layer:
    # W1*X + b1 ---relu---> W2*Y + b2
    def network(data):
        inp_la = tf.matmul(data, input_layer['weights'])\
         + input_layer['biases']
        if data.name=='train:0':
                relu = tf.nn.relu(tf.nn.dropout(inp_la,tf_probability))
        else:
            relu = tf.nn.relu(inp_la) 
        logits = tf.matmul(relu, hidden_layer['weights'])\
         + hidden_layer['biases']
        return logits
   
        
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels))
  
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

lista = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    #generate a random probability
    probability =  min(abs(np.random.normal(0.1, 0.1)),0.78)
    lista.append(probability)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_probability: probability}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print(sum(lista)/len(lista))
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))

Initialized
0.4012706192228854
Minibatch loss at step 0: 515.743896
Minibatch accuracy: 7.0%
Validation accuracy: 28.2%
0.11605666137101489
Minibatch loss at step 500: 250986879482238075735638016.000000
Minibatch accuracy: 24.2%
Validation accuracy: 48.9%
0.11904989930400235
Minibatch loss at step 1000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
0.11785856129836886
Minibatch loss at step 1500: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
0.1172873343635563
Minibatch loss at step 2000: nan
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
0.11674737559284713
Minibatch loss at step 2500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
0.11679234340075052
Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%


---
I did some tests with random probabilities
---------

In each iteration we pick a probability from a random distribution with mean mu and standart deviation sigma
if the value is greater then 1 we pick the probability 0.99

0) mu = 0.8, sigma = 0.2 -- Test accuracy: 87.5%

1) mu = 0.8, sigma = 0.3 -- Test accuracy: 86.9%

2) mu = 0.8, sigma = 0.1 -- Test accuracy: 88.7%

3) mu = 0.9, sigma = 1 -- Test accuracy: 83.5%

4) mu = 0.9, sigma = 0.3 -- Test accuracy: 87%

5) mu = 0.9, sigma = 0.1 -- Test accuracy: 88.5%

6) mu = 0.2, sigma = 2 -- Test accuracy: 88.5%

7) min(abs(np.random.normal(0.2, 2)),0.78) -- Test accuracy: 82.6%

8) min(abs(np.random.normal(0.1, 1)),0.78) -- Test accuracy: 85.4%

9) min(abs(np.random.normal(0.1, 0.1)),0.78) -- Test accuracy: 10.0%
  
  
  

----remember the normal NN have a test accuracy of 88.5%
  
  
  

In [58]:
#DROPOUTFUNCTION WITH L2

batch_size = 128

hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size \
                                        * image_size),name='train')
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_beta = tf.constant(0.001)
    
    #variables: input layer  
    input_layer = {'weights': \
    tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
    'biases': tf.Variable(tf.zeros([hidden_nodes]))}

    #hidden layer
    hidden_layer = {'weights':\
    tf.Variable(tf.truncated_normal([hidden_nodes,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
 
    #This is the network, given a dataset X it will return
    #the score after the hidden layer:
    # W1*X + b1 ---relu---> W2*Y + b2
    def network(data):
        inp_la = tf.matmul(data, input_layer['weights'])\
         + input_layer['biases']
        if data.name=='train:0':
                probability = tf.constant(0.01, dtype='float32')
                relu = tf.nn.dropout(tf.nn.relu(inp_la),probability)
        else:
            relu = tf.nn.relu(inp_la) 
        logits = tf.matmul(relu, hidden_layer['weights'])\
         + hidden_layer['biases']
        return logits
   
        
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels)) \
        + tf_beta*tf.nn.l2_loss(input_layer['weights'])\
        + tf_beta*tf.nn.l2_loss(hidden_layer['weights'])
  
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3202.104004
Minibatch accuracy: 13.3%
Validation accuracy: 9.3%
Minibatch loss at step 500: nan
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%


---
Tests with the dropout function and L2:
---------

0) With probability 0.7 we have a test accuracy of 92.0%

1) With probability 0.75 we have a test accuracy of 92.2%

2) With probability 0.8 we have a test accuracy of 92.4%

3) With probability 0.5 we have a test accuracy of 91.5%

4) With probability 0.99 we have a test accuracy of 92.8%

5) With probability 0.2 we have a test accuracy of 87.1%

6) With probability 0.01 we have a test accuracy of 10%




----remember the normal NN have a test accuracy of 88.5%

----remember using only L2 normalization we have a test accuracy of 92.6%



In [5]:

batch_size = 128

hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size \
                                        * image_size),name='train')
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))
    
    #probability to be used in the dropout function 
    tf_probability = tf.placeholder(tf.float32, shape=())

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_beta = tf.constant(0.001)
    
    #variables: input layer  
    input_layer = {'weights': \
    tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
    'biases': tf.Variable(tf.zeros([hidden_nodes]))}

    #hidden layer
    hidden_layer = {'weights':\
    tf.Variable(tf.truncated_normal([hidden_nodes,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
 
    #This is the network, given a dataset X it will return
    #the score after the hidden layer:
    # W1*X + b1 ---relu---> W2*Y + b2
    def network(data):
        inp_la = tf.matmul(data, input_layer['weights'])\
         + input_layer['biases']
        if data.name=='train:0':
                relu = tf.nn.relu(tf.nn.dropout(inp_la,tf_probability))
        else:
            relu = tf.nn.relu(inp_la) 
        logits = tf.matmul(relu, hidden_layer['weights'])\
         + hidden_layer['biases']
        return logits
   
        
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels)) \
        + tf_beta*tf.nn.l2_loss(input_layer['weights'])\
        + tf_beta*tf.nn.l2_loss(hidden_layer['weights'])
  
    #Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001
lista = []
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    probability =  min(abs(np.random.normal(0.2, 3)),0.99)
    lista.append(probability)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_probability: probability}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print(sum(lista)/len(lista))
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))

Initialized
0.99
Minibatch loss at step 0: 729.470215
Minibatch accuracy: 13.3%
Validation accuracy: 25.2%
0.859335704094899
Minibatch loss at step 500: 285.190399
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
0.8550266304478666
Minibatch loss at step 1000: 188.592407
Minibatch accuracy: 82.8%
Validation accuracy: 79.5%
0.8568935742344048
Minibatch loss at step 1500: 89.003540
Minibatch accuracy: 80.5%
Validation accuracy: 79.4%
0.859517561803324
Minibatch loss at step 2000: 57.126694
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
0.859348129324255
Minibatch loss at step 2500: 36.381504
Minibatch accuracy: 78.1%
Validation accuracy: 82.0%
0.8570091840009751
Minibatch loss at step 3000: 20.934921
Minibatch accuracy: 82.8%
Validation accuracy: 82.8%
Test accuracy: 89.0%


---
Tests with random probabilities and L2
---------

In each iteration we pick a probability from a random distribution with mean mu and standart deviation sigma
if the value is greater then 1 we pick the probability 0.99

0) mu = 0.8, sigma = 0.2 -- Test accuracy: 92.1%

1) mu = 0.8, sigma = 1 -- Test accuracy: 84.6%

2) mu = 0.2, sigma = 3 -- Test accuracy: 88.7%


 

----remember the normal NN have a test accuracy of 88.5%
----remember using only L2 normalization we have a test accuracy of 92.6%
  
  
  

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [6]:
batch_size = 128

hidden_nodes_1 = 60
hidden_nodes_2 = 40
hidden_nodes_3 = 20


graph = tf.Graph()
with graph.as_default():
    
    #placeholders for the SGD
    tf_train_dataset = tf.placeholder(tf.float32,\
        shape=(batch_size, image_size* image_size))
    
    tf_train_labels = tf.placeholder(tf.float32, \
        shape=(batch_size, num_labels))

    #constants: we use then to see accuracity of the network
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #constant for the L2 regularization 
    tf_beta = tf.constant(0.005)

    #input layer - Xavier initialization    
    tf_input_layer = {'weights': \
    tf.get_variable("weights_in", shape=[image_size * image_size, hidden_nodes_1],
           initializer=tf.contrib.layers.xavier_initializer())    ,
    'biases': tf.Variable(tf.zeros([hidden_nodes_1]))}

    #hidden layer 1  - Xavier initialization 
    tf_hidden_layer_1 = {'weights':\
    tf.get_variable("weights1", shape=[hidden_nodes_1, hidden_nodes_2],
                    initializer=tf.contrib.layers.xavier_initializer()),
    #tf.Variable(tf.truncated_normal([hidden_nodes_1,hidden_nodes_2])),
    'biases': tf.Variable(tf.zeros([hidden_nodes_2]))}
    
    #hidden layer 2 - normal initialization 
    tf_hidden_layer_2 = {'weights':\
    #tf.get_variable("weights2", shape=[hidden_nodes_2, hidden_nodes_3],
    #                initializer=tf.contrib.layers.xavier_initializer()),
    tf.Variable(tf.truncated_normal([hidden_nodes_2,hidden_nodes_3])),
    'biases': tf.Variable(tf.zeros([hidden_nodes_3]))}


    #hidden layer 3 - normal initialization 
    tf_hidden_layer_3 = {'weights':\
    #tf.get_variable("weights3", shape=[hidden_nodes_3, num_labels],
    #                initializer=tf.contrib.layers.xavier_initializer()),
    tf.Variable(tf.truncated_normal([hidden_nodes_3,num_labels])),
    'biases': tf.Variable(tf.zeros([num_labels]))}
    
    #L2 regularization
    def L2(beta,layer):
        return tf_beta*tf.nn.l2_loss(layer['weights'])
    
    
    #linear function of layer layer using data as input 
    def linear_activation(data,layer):
        return tf.matmul(data, layer['weights']) + layer['biases']  
 
    #This is the network: activation throught layers
    #with sigmoid function
    def network(data):
        input_layer = linear_activation(data, tf_input_layer) 
        sig_1 = tf.sigmoid(input_layer)
        hidden_layer_1 = linear_activation(sig_1,tf_hidden_layer_1)
        relu = tf.nn.relu(hidden_layer_1)
        hidden_layer_2 = linear_activation(relu,tf_hidden_layer_2)
        sig_2 = tf.sigmoid(hidden_layer_2)
        logits = linear_activation(sig_2,tf_hidden_layer_3)
        return logits
   
    
    #loss function that measures the distance between the network predictions
    #and the target labels
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(\
            network(tf_train_dataset), tf_train_labels))\
        #+ L2(tf_beta,tf_input_layer)\
        #+ L2(tf_beta,tf_hidden_layer_1)\
        #+ L2(tf_beta,tf_hidden_layer_2)\
        #+ L2(tf_beta,tf_hidden_layer_3)


    #Optimizer.
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.9
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.96, staircase=True)
    

    optimizer = (
    tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step))
    #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(network(tf_train_dataset))
    valid_prediction = tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))
   
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
   
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % \
            accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % \
    accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.587226
Minibatch accuracy: 4.7%
Validation accuracy: 12.4%
Minibatch loss at step 500: 0.740875
Minibatch accuracy: 78.9%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 0.390043
Minibatch accuracy: 88.3%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.422260
Minibatch accuracy: 88.3%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.471357
Minibatch accuracy: 84.4%
Validation accuracy: 85.4%
Minibatch loss at step 2500: 0.481553
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 0.461254
Minibatch accuracy: 84.4%
Validation accuracy: 86.3%
Test accuracy: 91.8%
